In [1]:
from datasets import load_dataset

dataset = load_dataset('TheNoob3131/mosquito-data')

Using custom data configuration TheNoob3131--mosquito-data-bf1f4d4d5f427763
Reusing dataset csv (C:\Users\Aswin.Surya24\.cache\huggingface\datasets\TheNoob3131___csv\TheNoob3131--mosquito-data-bf1f4d4d5f427763\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset['train'].features['Photo']
from datasets import Image
dataset = dataset.cast_column('Photo', Image(decode=True))

In [3]:
dataset = dataset.class_encode_column('Classification')
dataset = dataset.rename_column('Classification', 'labels')

Parameter 'function'=<function Dataset.class_encode_column.<locals>.cast_to_class_labels at 0x0000015038427550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Casting to class labels:   0%|          | 0/8 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
labels = dataset["train"].features["labels"].names
print(labels)

['aedes', 'culex', 'neither']


In [5]:
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
print(id2label)


{0: 'aedes', 1: 'culex', 2: 'neither'}


In [6]:
from transformers import AutoFeatureExtractor, ResNetForImageClassification
model_name_or_path = 'microsoft/resnet-18'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
print(feature_extractor)

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [7]:
from torchvision.transforms import (
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

transform = Compose(
    [
     RandomResizedCrop(feature_extractor.size),
     RandomHorizontalFlip(),
     ToTensor(),
     normalize
    ]
)

def train_transforms(examples):
  examples["pixel_values"] = [transform(image.convert("RGB")) for image in examples["Photo"]]

  return examples


In [8]:
prepared_ds = dataset.with_transform(train_transforms)

In [9]:
prepared_ds["train"][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [10]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.tensor([example["labels"] for example in examples])

  return {"pixel_values": pixel_values, "labels": labels}

dataloader = DataLoader(prepared_ds["train"], collate_fn=collate_fn, batch_size=4, shuffle=True)

In [11]:
batch = next(iter(dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [12]:
len(labels)

3

In [13]:
from transformers import ResNetForImageClassification
#print(len(labels))
model = ResNetForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model.train()

Downloading:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-18 and are newly initialized because the shapes did not match:
- classifier.1.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([3, 512]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [14]:
prepared_ds["train"][0]

{'MHM Id': 17233.0,
 'labels': 0,
 'Container': 'plant clumps (bamboo etc)',
 'Userid': 2538037,
 'latitude': -22.8854,
 'longitude': 47.7708,
 'date': '9/19/2019',
 'Photo': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4160x3120 at 0x1503D913C40>,
 'pixel_values': tensor([[[-2.0323, -2.0323, -2.0665,  ..., -2.0323, -1.9980, -1.9295],
          [-2.0494, -2.0494, -2.0152,  ..., -1.9980, -1.9809, -1.9467],
          [-2.0494, -2.0665, -2.0494,  ..., -1.9980, -1.9638, -1.9467],
          ...,
          [ 1.2899,  1.2728,  1.3584,  ..., -1.9980, -2.0152, -1.9980],
          [ 1.4098,  1.3755,  1.3755,  ..., -1.9809, -1.9980, -2.0152],
          [ 1.4269,  1.4440,  1.4098,  ..., -1.9295, -1.9467, -1.9638]],
 
         [[-1.9132, -1.9132, -1.9307,  ..., -2.0182, -2.0182, -2.0007],
          [-1.9132, -1.9132, -1.8782,  ..., -1.9832, -2.0007, -2.0007],
          [-1.8606, -1.8957, -1.8606,  ..., -1.9832, -1.9832, -2.0007],
          ...,
          [ 1.6057,  1.5882,  1.6057,  ..., 

In [15]:
from datasets import load_metric
import numpy as np


def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
  

    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./resnet-18-mosquito",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [17]:
prepared_ds['train'].features

{'MHM Id': Value(dtype='float64', id=None),
 'labels': ClassLabel(num_classes=3, names=['aedes', 'culex', 'neither'], id=None),
 'Container': Value(dtype='string', id=None),
 'Userid': Value(dtype='int64', id=None),
 'latitude': Value(dtype='float64', id=None),
 'longitude': Value(dtype='float64', id=None),
 'date': Value(dtype='string', id=None),
 'Photo': Image(decode=True, id=None)}

In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

In [19]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

C:\Users\Aswin.Surya24\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7107
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1780


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,0.873400,1.202113,0.560909,0.433556,0.476824,0.433556
200,0.817300,0.922775,0.549239,0.578947,0.552311,0.578947
300,0.844300,0.986800,0.564251,0.553068,0.548708,0.553068
400,0.734400,0.901893,0.562246,0.589706,0.567614,0.589706
500,0.755700,0.958425,0.604317,0.607153,0.563995,0.607153
600,0.784900,0.917097,0.566555,0.567898,0.567219,0.567898
700,0.889200,0.926778,0.582799,0.519337,0.535975,0.519337
800,0.716600,0.903834,0.580101,0.563245,0.568194,0.563245
900,0.624600,0.964241,0.600847,0.567607,0.581347,0.567607
1000,0.671400,0.927726,0.586895,0.580692,0.583524,0.580692


***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./resnet-18-mosquito\checkpoint-100
Configuration saved in ./resnet-18-mosquito\checkpoint-100\config.json
Model weights saved in ./resnet-18-mosquito\checkpoint-100\pytorch_model.bin
Feature extractor saved in ./resnet-18-mosquito\checkpoint-100\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./resnet-18-mosquito\checkpoint-200
Configuration saved in ./resnet-18-mosquito\checkpoint-200\config.json
Model weights saved in ./resnet-18-mosquito\checkpoint-200\pytorch_model.bin
Feature extractor saved in ./resnet-18-mosquito\checkpoint-200\preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8
Saving model checkpoint to ./resnet-18-mosquito\checkpoint-300
Configuration saved in ./resnet-18-mosquito\checkpoint-300\config.json
Model weights saved in ./resnet-18-mosquito\checkpoint-300

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 267288987GF
  train_loss               =      0.6989
  train_runtime            =  9:29:10.17
  train_samples_per_second =       0.832
  train_steps_per_second   =       0.052


In [20]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3439
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.5967
  eval_f1                 =     0.5756
  eval_loss               =     0.9004
  eval_precision          =     0.6034
  eval_recall             =     0.5967
  eval_runtime            = 0:20:15.60
  eval_samples_per_second =      2.829
  eval_steps_per_second   =      0.354
